In [1]:
import pandas as pd 
import numpy as np
import pandasql as ps
import os
from preprocess_days_stats import preprocess_cumulative_stats, preprocess_match_days
import warnings
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,accuracy_score
from preprocess_features import preprocess_columns
import tensorflow as tf 
from tensorflow.keras import layers
from helper_functions_tensorflow import CSVLoggerCallback, CSVLoggerCallbackParams
import random 
import numpy as np

# Ignora tutti i warning temporaneamente
warnings.filterwarnings("ignore")


df_giornate = preprocess_match_days(r"c:\Users\Hp\Documents\Serie_A_dump\csv_serie_a")

Reading file: I1 (0).csv
Reading file: I1 (1).csv
Reading file: I1 (10).csv
Reading file: I1 (11).csv
Reading file: I1 (12).csv
Reading file: I1 (13).csv
Reading file: I1 (14).csv
Reading file: I1 (15).csv
Reading file: I1 (16).csv
Reading file: I1 (17).csv
Reading file: I1 (18).csv
Reading file: I1 (2).csv
Reading file: I1 (3).csv
Reading file: I1 (4).csv
Reading file: I1 (5).csv
Reading file: I1 (6).csv
Reading file: I1 (7).csv
Reading file: I1 (8).csv
Reading file: I1 (9).csv
preprocessing finished!


In [31]:
import csv

# Nome del file CSV
nome_file = "results/baseline_KNN.csv"

# Apri il file CSV in modalità scrittura
with open(nome_file, mode='w', newline='') as file_csv:
    # Creazione dell'oggetto writer
    writer = csv.writer(file_csv)

    # Scrivi l'intestazione nel file CSV
    writer.writerow(['giorni_cumulativi', 'vanumero_colonne', 'vicini', 'val_accuracy', 'train_accuracy'])

# KNN baseline

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,accuracy_score
from preprocess_features import preprocess_columns

for giorni_cumulativi in range(10):
    giorni_cumulativi = giorni_cumulativi+1
    df_Serie_A_days, stats_teams_serie_A = preprocess_cumulative_stats(dataframe=df_giornate, giorni_cumulativi=giorni_cumulativi)
    for numero_colonne in ['all','less','few']:    
        X_train_norm, X_valid_norm, X_test_norm, Train_labels_encoded, Valid_labels_encoded, Test_labels_encoded = preprocess_columns(
            df_Serie_A_days,numero_colonne,giorni_cumulativi)

        best_accuracy=0
        for vicini in range(3,40): #prendiamo diversi numeri di vicini possibili
            knn_model = KNeighborsClassifier(n_neighbors=3*vicini) #addestriamo il modello con questo numero di vicini 
            knn_model.fit(X_train_norm, Train_labels_encoded) 
            y_valid = knn_model.predict(X_valid_norm)
            val_accuracy = round(accuracy_score(Valid_labels_encoded, y_valid),2)
            y_train = knn_model.predict(X_train_norm)
            train_accuracy = round(accuracy_score(Train_labels_encoded, y_train),2)

            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                
                # Scrivi i dati nel file CSV
                with open(nome_file, mode='a', newline='') as file_csv:
                    writer = csv.writer(file_csv)
                    writer.writerow([giorni_cumulativi, numero_colonne, vicini*3, val_accuracy, train_accuracy])            

In [ ]:
baseline_KNN_df = pd.read_csv(r'results/baseline_KNN.csv').sort_values(['val_accuracy'],ascending=False)
baseline_KNN_df.head(10)

,giorni_cumulativi,vanumero_colonne,vicini,val_accuracy,train_accuracy
109,6,less,105,0.51,0.51
16,1,few,45,0.51,0.52
116,6,few,78,0.50,0.51
11,1,less,84,0.50,0.51
23,2,all,72,0.50,0.51
29,2,less,42,0.50,0.51
108,6,less,39,0.50,0.52
188,10,less,48,0.50,0.52
15,1,few,39,0.50,0.52
96,5,few,111,0.50,0.50


In [ ]:
baseline_KNN_df.sort_values(['train_accuracy'],ascending=False).head(10)

,giorni_cumulativi,vanumero_colonne,vicini,val_accuracy,train_accuracy
118,7,few,9,0.39,0.58
0,1,all,9,0.46,0.58
150,9,few,9,0.44,0.58
44,3,few,9,0.44,0.58
10,1,few,9,0.48,0.58
24,2,few,9,0.45,0.57
35,3,less,9,0.41,0.57
105,7,all,9,0.39,0.57
19,2,less,9,0.44,0.57
15,2,all,9,0.46,0.57


# Dense neural network

In [ ]:
import tensorflow as tf 

# Create a function to implement a ModelCheckpoint callback with a specific filename 
def create_model_checkpoint(model_name, save_path="model_experiments"):
  return tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(save_path, model_name), # create filepath to save model
                                            verbose=0, # only output a limited amount of text
                                            monitor='val_loss',
                                            save_best_only=True) # save only the best model to file

## Iniziamo con poche features e 3 giorni cumulativi

In [2]:
#set the seed
tf.random.set_seed(42)
np.random.seed(42)
random.seed(12)

In [4]:
def create_and_compile_dense(num_dense_layers, first_dropout, other_dropouts, first_num_neurons,
                              other_num_neurons, input_shape, first_activation='relu', other_activations='relu'):

    inputs = layers.Input(shape=(input_shape,))
    x = layers.Dense(first_num_neurons, activation=first_activation)(inputs)
    x = layers.Dropout(first_dropout)(x)  # Aggiunto il layer di dropout per ridurre overfitting
    for i in range(num_dense_layers-1):
        x = layers.Dense(other_num_neurons, activation=other_activations)(x)
        x = layers.Dropout(other_dropouts)(x) 
    outputs = layers.Dense(3, activation='softmax')(x)
    dense_model = tf.keras.Model(inputs, outputs, name='model_1_dense')

    # compilo il modello 
    dense_model.compile(
        loss='sparse_categorical_crossentropy',
        metrics = 'accuracy',
        optimizer = tf.keras.optimizers.Adam()
    )

    return dense_model

In [8]:
for giorni_cumulativi in [1,3,5]:
    giorni_cumulativi = giorni_cumulativi+1
    # calcolo il dataframe con i giorni cumulativi 
    df_Serie_A_days, stats_teams_serie_A = preprocess_cumulative_stats(dataframe=df_giornate, giorni_cumulativi=giorni_cumulativi)
    print(f'giorni cumulativi: {giorni_cumulativi}')
    # seleziono il numero di features da usare nel modello 
    for numero_colonne in ['all','less','few']:
        print(f'numero colonne: {numero_colonne}')    
        X_train_norm, X_valid_norm, X_test_norm, Train_labels_encoded, Valid_labels_encoded, Test_labels_encoded = preprocess_columns(
            df_Serie_A_days,numero_colonne,giorni_cumulativi)
        input_shape = X_train_norm.shape[1]
        # fast preprocessing 
        X_train_norm = tf.data.Dataset.from_tensor_slices((X_train_norm, Train_labels_encoded))
        X_valid_norm = tf.data.Dataset.from_tensor_slices((X_valid_norm, Valid_labels_encoded))
        X_train_norm = X_train_norm.batch(32).prefetch(tf.data.AUTOTUNE) #Autotune è per dirgli di prefetchare tanti dati quanti può
        X_valid_norm = X_valid_norm.batch(32).prefetch(tf.data.AUTOTUNE)
        #iniziamo a creare i modelli 
        for num_dense_layers in [2,3,4]:
            for first_dropout in [0.0, 0.2, 0.4, 0.6]:
                for other_dropouts in [0.0, 0.2, 0.4, 0.6]:
                    for first_num_neurons in [32,64,128]:
                        for other_num_neurons in [8, 16, 32]:
                            other_activations = 'relu'
                            first_activation = 'relu'
                            model_1 = create_and_compile_dense(num_dense_layers, first_dropout, other_dropouts, first_num_neurons,
                                    other_num_neurons, input_shape, first_activation, other_activations)

                                # fitto il modello 
                            model_1.fit(
                                    x = X_train_norm,
                                    epochs = 15,
                                    verbose=0,
                                    validation_data = X_valid_norm,
                                    callbacks = [
                                        CSVLoggerCallbackParams('results.csv','model_1_dense', num_dense_layers, first_dropout, other_dropouts, first_num_neurons,
                                                                    other_num_neurons, first_activation, other_activations), 
                                        #create_model_checkpoint(model_name= 'Dense_model_best')
                                                ] 
                                )

preprocess finished
giorni cumulativi: 2
numero colonne: all
utilizzando tutte le features


KeyboardInterrupt: 

In [ ]:
# model_1 = loaded_model = tf.keras.models.load_model("model_experiments\Dense_model_best")
# model_1.evaluate(X_test_norm, Test_labels_encoded)

22/22 [==============================] - 0s 1ms/step - loss: 1.0116 - accuracy: 0.5115


[1.0115653276443481, 0.5114942789077759]